# <b>Modelo de programación MapReduce (Python)</b>
## <i>Big Data Analytics</i>

Curso 2023/24

Prof. *Dr. José Raúl Romero Salguero*

---



Vamos a ver en este notebook una introducción al desarrollo con el modelo de programación MapReduce en Python.

Al desarrollarse sobre Google Colab, no se implementa para un entorno clúster, si bien sí que se respetará la filosofía del paradigma.

Para conocer el modelo, tras la instalación del entorno, comenzaremos definiendo las funciones `map` y `reduce` de Python.

---

# **Instalación del entorno**
## Instalación de Hadoop

Instalamos la versión de Hadoop/Spark 3.2.4
Se recomienda visitar el sitio de Apache Spark para descargar esta versión:

https://spark.apache.org/downloads.html

Se configuran posteriormente las variables de entorno `JAVA_HOME` y `SPARK_HOME`

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.4-bin-hadoop3.2"

La descarga de Hadoop puede tomar su tiempo, según la conexión disponible. Se borra posteriormente de la máquina virtual el archivo `.tgz`

In [ ]:
# Descomentar las líneaa según la necesidad
!wget https://archive.apache.org/dist/spark/spark-3.2.4/spark-3.2.4-bin-hadoop3.2.tgz
!tar -xf spark-3.2.4-bin-hadoop3.2.tgz
#!rm spark-3.2.4-bin-hadoop3.2.tgz

--2023-12-14 19:20:13--  https://archive.apache.org/dist/spark/spark-3.2.4/spark-3.2.4-bin-hadoop3.2.tgz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189, 2a01:4f9:1a:a084::2
Connecting to archive.apache.org (archive.apache.org)|65.108.204.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 301183180 (287M) [application/x-gzip]
Saving to: ‘spark-3.2.4-bin-hadoop3.2.tgz’

spark-3.2.4-bin-had 100%[===================>] 287.23M  11.7MB/s    in 26s     

2023-12-14 19:20:41 (10.9 MB/s) - ‘spark-3.2.4-bin-hadoop3.2.tgz’ saved [301183180/301183180]



## Instalación e iniciación de la sesión de Spark

* Buscamos la librería `findspark` con `pip install`


In [ ]:
!pip install findspark

* Con `SparkSession` inicializamos

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local[*]")\
        .appName("Spark_Dataframes")\
        .getOrCreate()

In [ ]:
spark

# ***MapReduce*** en Python

* *MapReduce* es un **modelo de programación en paralelo escalable**, esto es, concebido para operar con grandes volúmenes de datos en grandes clústers.
* La **infraestructura**/plataforma (p.ej. Apache Hadoop, Apache Spark, etc.) se encarga de ocultar al programador operaciones de los sistemas distribuidos como el balanceo de carga, tráfico en red, optimización de transferencia en disco, serialización de los datos, gestión de fallos de máquinas/transferencia de datos, etc.
  * Hadoop está escrito en Java, y Spark en Scala
  * Hadoop permite ser desarrollado en otros lenguajes, como Python, para lo que se debe transformar el código en un `JAR` de Java, p.ej. utilizando [Jython](https://www.jython.org/).
  * Spark ofrece interfaz/librería para el uso de Python (pyspark [texto del enlace](https://spark.apache.org/docs/latest/api/python/))
* La **programación funcional** de Python, vista en el notebook anterior, encaja muy bien en el modelo de programación, si bien no es de uso obligado.


## **Recordemos** los pasos habituales del modelos de programación *MapReduce*

1. Extracción, transformación y carga de un gran conjunto de datos.
2. Operación `map`
3. Combinar (*shuffle*) y ordenar → localización de tareas en nodos
4. Operación `reduce` (resumen, filtrado y transformación)
5. Escritura de resultados

Las funciones `map` y `reduce` pueden trabajar en paralelo sobre distintas claves o distintos elementos de la colección de datos.

## Ejemplo de conteo de palabras (*Word count*)

Es un ejemplo ilustrativo muy utilizado por Apache Foundation para explicar el funcionamiento de MapReduce. El **problema** consiste en contar cada una de las palabras que aparecen en un conjunto de documentos (identificados por su URI).

En primer lugar, debemos **formalizar el formato de las tuplas** `<clave, valor>` con las que trabajará la función `map`.

* key: URI
* value: Contenido del documento

$\left< shakesp1, to \; be \; or \; not \; to \; be \right>$

En segundo lugar, la **función `map`** obtendrá los pares de conteo individual de cada una de las palabras del conjunto de datos que recibe ese `map`.

>  $\left< to, 1 \right>$
  $\left< be, 1 \right>$
  $\left< or, 1 \right>$
  $\left< not, 1 \right>$
  $\left< to, 1 \right>$
  $\left< be, 1 \right>$

En tercer lugar, dada la simplicidad de la **operación `reduce`**, es la función reduce la que agrupa los valores conforme a su clave y realiza la suma.

>  $\left< be, 2 \right>$  
  $\left< not, 1 \right>$  
  $\left< or, 1 \right>$  
  $\left< to, 2 \right>$  


El **procedimiento general** a seguir para cualquier problema:

- La función `map` agrupa los valores según su *`key`*, y luego invoca a un `reduce` para cada clave.

  - Para ello, las colecciones se dividen en diferentes unidades de almacenamiento.

  - MapReduce particionará los datos minimizando la copia de estos en el clúster.

- Los datos de las diferentes particiones se reducen por separado y en paralelo.

- El resultado final de la función `reduce` es una reducción de los datos ya reducidos en cada partición individual.

  - Recordemos que, para que funcione, **el operador debe ser conmutativo y asociativo**.

  - En el caso de esta función, el operador es `+`.

---

**NOTA**: A diferencia de `map` / `reduce` de Python estándar, en este caso la función `reduce` funciona exclusivamente con pares $\left< key, value\right>$. Algunos autores, como Phelps proponen nombrarla **`ReduceByKey`** (`reducirPorClave`)

---



### Implementación no escalable, no paralela

Tomada de (Phelps, 2016), desarrollamos un *ejemplo no paralelo y no escalable* del problema planteado. Por tanto, es importante considerar que **esta no es la forma** en la que Hadoop o Spark lo implementan. El **objetivo** es mostrar cómo funciona el modelo de programación MapReduce y cómo un ejemplo sencillo puede ser codificado en términos de operaciones `map` y `reduce`.

En primer lugar, codificamos las funciones de combinar (agrupar por clave - *`groupByKey`* o `agruparPorClave`) y reducir por clave (*`reduceByKey`* o `reducirPorClave`):

In [ ]:
from functools import reduce

def agruparPorClave(data):
  # Devolvemos un conjunto de pares <clave, valor(es)>
  res = dict()
  # Para cada <clave, valor> en los datos (previsiblemente de map)
  for clave, valor in data:
    if clave in res:
      # Si la clave ya existe en el diccionario, se agrega otro valor a esa misma clave
      res[clave].append(valor)
    else:
      # Si no existe, se crea nueva
      res[clave] = [valor]
  # Se devuelve la nueva agrupación
  return res

def reducirPorClave(fn, data):
  # Lo primero para reduce es agrupar por la misma clave
  pares = agruparPorClave(data)
  # Como reducción, se devolverá una colección de pares <clave, valor'>, donde
  # valor' es el resultado de aplicar fn a la lista de valores con esa clave.
  return list(map(lambda clave:
                  (clave, reduce(fn, pares[clave])),
                  pares))

Podemos particularizar el código anterior aplicándolo a un caso concreto del problema *WordCount*. Continuamos con el ejemplo que se inició arriba:

In [ ]:
# La función de mapeo consiste en generar pares (x, 1) para cada palabra que se encuentra en el conjunto de datos
data = list(map(lambda x: (x, 1), "to be or not to be".split()))
print(data)

[('to', 1), ('be', 1), ('or', 1), ('not', 1), ('to', 1), ('be', 1)]


In [ ]:
# Estos pares deben ser agrupados por clave
agruparPorClave(data)
# NOTA: Esta llamada es solo a modo ilustrativo (por eso no se asigna a nada), ya que se llama en la siguiente invocación de la operación reduce()

{'to': [1, 1], 'be': [1, 1], 'or': [1], 'not': [1]}

In [ ]:
reducirPorClave(lambda x,y: x + y, data)

[('to', 2), ('be', 2), ('or', 1), ('not', 1)]

### Implementación paralela basada en hilos (no Spark)

La implementación paralela se basa en la codificación `fn_map_multihilo` que se explicó en el *notebook 02*.

Esta sección expone cómo realizar computaciones MapReduce que exploten el paralelismo que ofrecen los múltiples *cores* de una única computadora. Para ello, primero rescatamos la definición de `fn_map_multihilo`:

In [ ]:
## CÓDIGO EXPLICADO EN Notebook 02
from threading import Thread

# Descomentar print para hacer seguimiento de la ejecución o eliminar comentarios print para limpiar código

def hacer_hilo(fn, res, data, hilos, i):
    # La evaluación de cada función se planifica en un core distinto
    def trabajo():
        #print("trabajo(", threading.current_thread().name, "): Procesando datos:", data[i])
        # Es realmente en este punto donde se aplica lambda sobre los datos concretos del hilo
        res[i] = fn(data[i])
        #print("trabajo(", threading.current_thread().name, "): Finalizado hilo #", i)
        #print("trabajo(", threading.current_thread().name, "): Resultado es", res[i])
    hilos[i] = Thread(target=trabajo)

def fn_map_multihilo(fn, data):
    # El programa principal es el encargado de planificar los trabajos, ejecutar los hilos y esperar a que terminen
    n = len(data)
    res = [None] * n
    hilos = [None] * n
    #print("fn_map_multihilo(): Planificando trabajos")
    for i in range(n):
        hacer_hilo(fn, res, data, hilos, i)
    #print("fn_map_multihilo(): Iniciando trabajos")
    for i in range(n):
        hilos[i].start()
    #print("fn_map_multihilo(): Esperando terminación de hilos")
    for i in range(n):
        hilos[i].join()
    #print("¡Terminado!")
    return res

Con la función de mapeo multihilo ya definida, lo siguiente es hacer uso de ella en la versión anterior de `reducirPorClave`. Recuerda que `fn_map_multihilo` no se refiere a la función MapReduce, sino a la operación Python de programación funcional.

Redefinimos `reducirPorClave`:

In [ ]:
def reducirPorClave_multihilo(fn, data):
  pares = agruparPorClave(data)
  return fn_map_multihilo(lambda clave: (clave, reduce(fn, pares[clave])), [clave for clave in pares])

---

**NOTA**: En Python, `[expr for loop]` es la sintaxis para la comprensión de listas (***list comprehension***), que permite crear un lista en una única línea de código. Esta es una de las características distintivas más potentes de Python, pero debe utilizarse con cuidado por motivos de legibilidad y eficiencia.

Por ejemplo:

> `cuadrados = [i * i for i in range(10)]`

---

Continuamos con el ejemplo multihilo aplicado al conteo de palabras. Para ello, invocamos a `reducirPorClave_multihilo` con la función de suma y los datos de Shakespeare.

In [ ]:
reducirPorClave_multihilo(lambda x,y: x + y, data)

[('to', 2), ('be', 2), ('or', 1), ('not', 1)]

Pero si nos fijamos en el código de `reducirPorClave_multihilo`, el paso `reduce` de MapReduce no está realmente siendo paralelizado en términos de la distribución de datos que aplicaría MapReduce. Realmente, para **paralelizar la tarea `reduce`** debemos considerar:

- El operador debe ser conmutatitvo y asociativo.
- Los datos deben estar divididos en particiones de tamaño similar.
- Aplicamos la operación reduce a cada partición independientemente en un core separado.
- Los resultados se combinan al final del paso de reducción.

En primer lugar, implementamos la operación de partición de datos:

In [ ]:
def particionar_datos(data, ptos_particion):
    # Declaramos la lista de particiones
    particiones = []
    # Número del primer elemento de la siguiente partición
    n = 0
    for i in ptos_particion:
        # Agregamos a particiones subconjuntos del dataset
        particiones.append(data[n:i])
        n = i
    # Agregamos el último subconjunto
    particiones.append(data[n:])
    return particiones

En segundo lugar, creamos la función `reduce` para operar sobre múltiples particiones de datos de forma paralela:

In [ ]:
def reduce_paralelo(fn, particiones):
  num_part = len(particiones)
  res = [None] * num_part
  hilos = [None] * num_part

  def trabajo(i):
    res[i] = reduce(fn, particiones[i])

  for i in range(num_part):
    hilos[i] = Thread(target = lambda: trabajo(i))
    hilos[i].start()

  for i in range(num_part):
    hilos[i].join()

  print("Reducciones de hilos: ",res)
  return reduce(fn, res)

In [ ]:
# Supongamos un conjunto de datos que queremos fusionar en una única palabra:
datos = ['a', 'b', 'c', 'd', 'e', 'f', 'g']
# Que queremos particionar en 3 subconjuntos: a partir del 3er y 6o elemento
particiones = particionar_datos(datos, [2,5])
print("Particiones de datos: ", particiones)
# La reducción se hace por particiones de forma paralela y posteriormente se fusionan los resultados de las 3 particiones:
reduce_paralelo(lambda x,y: x + y, particiones)

Particiones de datos:  [['a', 'b'], ['c', 'd', 'e'], ['f', 'g']]
Reducciones de hilos:  ['ab', 'cde', 'fg']


'abcdefg'

# ***MapReduce*** en Spark

Recordemos que Spark es un superconjunto de MapReduce, extensión de Hadoop, también de Apache.

Provee objetos que representan los RDD (***Resilient Distributed Datasets***)

> Un RDD es la estructura de datos básica fundamental de Spark. Hay colecciones de datos distribuidos inmutables de cualquier tipado, constituyendo registros de datos tolerantes a fallos (resilientes) que residen en múltiples nodos.

**¿Cómo se comportan los RDD?** De forma similar a las listas de Python pero considerando que las colecciones son inmutables y que los datos se distribuyen por los nodos del clúster.

Al aplicar MapReduce, cada instancia de un RDD tiene declarados al menos dos métodos para el flujo de tareas:

- `map`
- `reduce` (→ `reducirPorClave`)

El funcionamiento de estos métodos es el mismo que el declarado arriba (solo que se hizo sobre colecciones estándares de Python).

## Conversión a RDD con el contexto de Spark: clase `SparkContext`

Cuando se trabaja con Spark, invocaremos a métodos de un objeto de `pyspark.context.SparkContext`, que representa al contexto de ejecución. Un nombre habitualmente utilizado para estas instancias es **`sc`**.

El **método `parallelize`** se utiliza para convertir una colección estándar de Python en un RDD. Lo más habitual es que este RDD se cree a partir de una tabla HBase o de un gran dataset.

Para ello, una vez inicializado el entorno, se crea el contexto de Spark.

In [ ]:
from pyspark import SparkContext
sc =SparkContext.getOrCreate()

Supongamos que queremos convertir una lista de Python en un RDD.

In [ ]:
palabras = "to be or not to be".split()
palabras_rdd = sc.parallelize(palabras)
palabras_rdd

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

Obsérvese que con la invocación a `map` o `reducirPorClave` sobre un RDD (p.ej. `palabras_rdd`) ya podemos configurar una operación de procesamiento paralelo distribuido por el clúster.


## Función `map` sobre un RDD



En Spark, los RDD implementan los métodos de `map` y `reduceByKey`.

In [ ]:
tuplas_palabras_rdd = palabras_rdd.map(lambda x: (x, 1))
tuplas_palabras_rdd

PythonRDD[1] at RDD at PythonRDD.scala:53

La operación anterior es lanzada en el clúster pero no se llevará a cabo la computación solicitada y, consecuentemente, no tendremos el resultado todavía hasta que solicitemos el resultado final invocando al **`método collect()`**.

In [ ]:
tuplas_palabras_rdd.collect()

[('to', 1), ('be', 1), ('or', 1), ('not', 1), ('to', 1), ('be', 1)]

**Recuerda**:
- Solo cuando invocamos a `collect` se realiza el procesamiento en el clúster.
- Si la colección resultante del mapeo es muy grande, entonces puede ser una operación muy costosa.

En caso de que sea muy costoso y queramos hacer pruebas o no nos interese el resultado completo por algún motivo, la **operación `take`** es similar a `collect` pero devuelve únicamente los primeros `n` elementos.

In [ ]:
tuplas_palabras_rdd.take(3)

[('to', 1), ('be', 1), ('or', 1)]

## Función `reduce` sobre un RDD

Una vez realizada la primera tarea de mapeo, necesitamos ejecutar las siguientes tareas del trabajo MapReduce. Procedemos de forma similar al mapeo con la reducción por clave. Para ello, utilizamos el **método `reduceByKey`**, cuyo funcionamiento interno hemos simulado previamente en los ejemplos anteriores.

In [ ]:
tuplas_palabras_rdd = tuplas_palabras_rdd.reduceByKey(lambda x,y: x + y)

Y solicitamos que se compute el resultado final:

In [ ]:
tuplas_palabras = tuplas_palabras_rdd.collect()
tuplas_palabras

[('to', 2), ('be', 2), ('or', 1), ('not', 1)]

## Ejemplo de conteo de palabras en MapReduce

Todo lo anterior se resume en que podemos implementar todas las tareas de MapReduce para el caso del conteo de palabras simplemente con el siguiente código (Phelps, 2016):

In [ ]:
text = "to be or not to be".split()
rdd = sc.parallelize(text)
counts = rdd.map(lambda word: (word, 1)) \
             .reduceByKey(lambda x, y: x + y)
counts.collect()

[('to', 2), ('be', 2), ('or', 1), ('not', 1)]

# Operaciones alternativas sobre un RDD

Anteriormente hemos visto como realizar el mapeo y reducción sobre clúster con un RDD. Sin embargo, en trabajos más complejos podemos necesitar otras operaciones adicionales que ayuden a la operación sobre colecciones de tuplas (propias del marco MapReduce).

Una operación alternativa es la **lectura del dataset desde un fichero de texto**, ya que no siempre disponemos del dataset en alguna de las formas que hemos visto hasta ahora - habitualmente basadas en colecciones de memoria. Para ello, utilizaremos el **método textFile**, que toma como argumento la URI del fichero, esto es, tanto un directorio local como un directorio remoto ("`hdfs://`", "`s3a://`", etc.). Además, el método `textFile` puede manejar directorios, comodines ("`directorio/*.txt`") e incluso ficheros comprimidos ("`genome.txt.gz`").

No obstante, consideremos que la forma más habitual en Big Data es que los datos se encuentren en ficheros HDFS o en tablas HBase.

***Ejemplo***: Deseamos convertir el contenido de un fichero texto, *genome.txt* (sistema de ficheros ext4), en un RDD, donde cada elemento se corresponde con una línea del mismo.

*NOTA*: Descargue el fichero `genome.txt` de Moodle y súbalo a su carpeta raíz del entorno actual de ejecución de Google Colab.

In [ ]:
genoma_rdd = sc.textFile("genome.txt")
genoma_rdd.take(3)

['TTGGCCATGCTGCCCACTCACCTAGAGCGCACAGCTGACACTGAGTCCTCTTCTGAACCTCATCCATGAA',
 'CATATTTATGAAATCTTTCCTGGCCCCAAGTGGAAATGCCCCCTCATTTGGGTCCTCACTGAACCCCAGT',
 'ACACAACTCTTTTGTACTACTCTATTATGCTGGGGTGTTTTTTTATTGTCTCACCTGATAAACCGTAAGC']

**Otras operaciones adicionales** que podremos necesitar, según el caso, son las siguientes:

- `filter(fn)` → Devuelve un nuevo dataset formado por la selección de aquellos elementos de la fuente en los que `fn` devuelve `True`.
- `sortByKey([ascending], [numPartitions])` → Cuando se invoca a un dataset de pares (K, V), donde K implementa `Ordered`, este método devuelve un conjunto de pares (K, V) ordenados por las claves en orden ascendente o descendente, según su argumento.
- `coalesce(numPartitions)` → Reduce el número de particiones de un RDD a `numPartitions`. Este método es especialmente útil cuando se ha realizado un filtrado y se ha reducido el tamaño de las particiones, por lo que se pueden transformar en menos para su operatividad más eficiente en el clúster.
- `count()` → Devuelve el número de elementos del dataset.
- `countByKey()` → Devuelve un *hashmap* de pares `(K, int)` con la cuenta de elementos asociados a cada clave `K`, sólo si es un RDD de tipo `(K, V)`.
- `saveAsTextFile(path)` → Escribe los elementos en un fichero de texto (o conjunto de ficheros de texto).

**NOTA**: La [guía de programación de RDD](https://spark.apache.org/docs/latest/rdd-programming-guide.html) de la web oficial de Spark contiene el listado completo de operaciones sobre estos elementos.

## *Ejemplo de uso de RDD*: Problema del genoma

Se pretende calcular -haciendo uso de RDD- la frecuencia de secuencias de 5 bases en una cadena de genoma.

En primer lugar, se crea el método que permite agrupar y separar en grupos de *n* elementos la cadena de genoma.

In [ ]:
def agruparCaracteres(linea, n=5):
    res = ''
    i = 0
    for c in linea:
        res = res + c
        i = i + 1
        if (i % n) == 0:
            yield res
            res = ''

def agrupar_partir(linea):
    return [sec for sec in agruparCaracteres(linea)]

En este caso, dado que el RDD es una colección compuesta de un elemento por línea, si utilizamos `map` devolvería un tipo de dato multidimensional, esto es, una lista por cada colección/línea leída del fichero. Por ello, tenemos que aplanar la estructura resultante del mapeado con el método `flatMap`.

Posteriormente, esta lista de bases (composición de 5 caracteres) deben transformarse en un RDD que contiene la forma $\left< K, V \right>$, donde la clave $K$ es la secuencia y el valor $V$ es `1`. Si recordamos, en el problema del conteo de palabras se operaba igual.

In [ ]:
secuencias = genoma_rdd.flatMap(agrupar_partir)

In [ ]:
# Primero, convertimos la secuencia en un conjunto de <K,V> en forma MapReduce
# Segundo, aplicamos la reducción por clave: suma de los valores individuales para la misma clave
conteo = secuencias.map(lambda k: (k, 1)).reduceByKey(lambda x,y: x + y)
conteo.take(10)

[('TTGGC', 587),
 ('CATGC', 647),
 ('TGCCC', 599),
 ('ACTCA', 775),
 ('TGACA', 831),
 ('TTCTG', 1257),
 ('AACCT', 726),
 ('TTATG', 819),
 ('AAATC', 996),
 ('TGGCC', 718)]

Para obtener los pares ordenados por el número de apariciones, podemos utilizar métodos nativos de RDD, como `sortByKey`. Puesto que en el par $\left< K, V \right>$ el conteo es el valor $V$, y no la clave $K$, El detalle está en que debemos revertir la clave y el valor para que ordene por este valor.

Para ello, desarrollamos una función que revierta clave y valor, y un mapeo que lo aplique. Finalmente, se podrá ordenar por la clave.

In [ ]:
def revertir_tupla(par):
  return (par[1], par[0])

# Se revierte la clave y el valor para poder ordenar por clave
secuencias = conteo.map(revertir_tupla)

# Se ordena por clave (descendiente: False)
secuencias_ord = secuencias.sortByKey(False)
secuencias_ord.take(5)

[(37137, 'NNNNN'),
 (4653, 'AAAAA'),
 (4223, 'TTTTT'),
 (2788, 'AAAAT'),
 (2658, 'ATTTT')]

Recordemos que nos piden la **frecuencia de aparición**. Por tanto, necesitaremos calcular todo el conteo de bases en las distintas secuencias. Si bien hay otros métodos de obtenerlos (*ver apuntes de teoría del tema*), podremos sumar las claves:

In [ ]:
# Suponemos que no necesitamos más operaciones considerando la base como K.
#     En este caso, habría que revertir el orden de nuevo (operación costosa)

# Primero, calculamos el total de bases
total = secuencias_ord.keys().sum()

# Aplicamos el cálculo de la frecuencia con comprensión de lista (list comprehension)
secuencias = [(sec[0]/total,sec[1]) for sec in secuencias_ord.collect()]

print("Número total de bases: ",total)
secuencias[:10]

Número total de bases:  699988


[(0.05305376663599948, 'NNNNN'),
 (0.006647256810116745, 'AAAAA'),
 (0.006032960565038258, 'TTTTT'),
 (0.003982925421578656, 'AAAAT'),
 (0.0037972079521363224, 'ATTTT'),
 (0.00326148448259113, 'AAATA'),
 (0.00325148431115962, 'TAAAA'),
 (0.003138625233575433, 'TTTTA'),
 (0.0031371966376566454, 'TATTT'),
 (0.0031214820825499865, 'AGAAA')]

# Finalización de la sesión de Spark

In [ ]:
spark.stop()

# <b>Referencias</b>

Estructura y contenido básico inicial adoptado de:
* S. Phelps, [*Data science and big data with Python*](https://github.com/phelps-sg/python-bigdata), 2016

Información adicional:

* Ejemplo [`WordCount` en Python](https://cwiki.apache.org/confluence/display/HADOOP2/PythonWordCount) (apache.org, 2019)
* V. Yordanov, [*Python Basics: Mutable vs Immutable Objects*](https://towardsdatascience.com/https-towardsdatascience-com-python-basics-mutable-vs-immutable-objects-829a0cb1530a#:~:text=Some%20of%20the%20mutable%20data,string%2C%20tuple%2C%20and%20range.), 2019
* C. Gaur, [*A Complete Guide to RDD in Apache Spark*](https://www.xenonstack.com/blog/rdd-in-spark/#:~:text=Resilient%20Distributed%20Dataset%20(RDD)%20is,that%20resides%20on%20multiple%20nodes.), 2020
* Tutorial básico/guía de referencia de Python: https://www.w3schools.com/python/
* Tutorial de Python: https://www.geeksforgeeks.org/python-programming-language/
* PySpark Documentation: https://spark.apache.org/docs/latest/api/python/
* RDD Programming Guide: https://spark.apache.org/docs/latest/rdd-programming-guide.html
* Introduction to Parallel Computing Tutorial: https://hpc.llnl.gov/documentation/tutorials/introduction-parallel-computing-tutorial